In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-126886")
exp = Experiment(workspace=ws, name="udacity-project1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-126886
Azure region: southcentralus
Subscription id: 8ca4d354-cab8-4c39-9d61-ea227d2daaad
Resource group: aml-quickstarts-126886


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name="cpu_cluster" #The compute cluster created before in the Azure portal
try:
    # Checking for existing compute target
    train_cluster = ComputeTarget(workspace=ws, name="cpu-cluster")
    print("Found existing cluster which we'll use instead of creating a new one.")
except ComputeTargetException:
    # If it doesn't exist, create a compute cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    train_cluster = ComputeTarget.create(ws, "train_cluster", compute_config)
    train_cluster.wait_for_completion(show_output=True)

Found existing cluster which we'll use instead of creating a new one.


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler
### YOUR CODE HERE ###

ps = RandomParameterSampling( {
        "--C":uniform(0.05,1),
        "--max_iter":choice(20,40,60,80,100)
    }
) 


# Specify a Policy
### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2,slack_factor=0.2)

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
from azureml.core import ScriptRunConfig
from azureml.core import Environment
script='train.py'
est=SKLearn(source_directory='.',entry_script=script,compute_target=train_cluster)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(estimator=est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=4,
                             max_concurrent_runs=4
                                    )

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run=exp.submit(hyperdrive_config,show_output=True )
RunDetails(hyperdrive_run).show() #Run history widget
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###